## Einrichtung von Julia JuMP
---
Überprüfen Sie die bereits installierten Pakete. In der JuliaBox sind möglicherweise alle notwendigen Pakete bereits vorhanden.

In [1]:
using Pkg;
Pkg.status()

    Status `C:\Users\Lotte\.julia\environments\v1.2\Project.toml`
  [60bf3e95] GLPK v0.12.0
  [7073ff75] IJulia v1.20.0
  [4076af6c] JuMP v0.20.0
  [f3b207a7] StatsPlots v0.12.0


Installieren Sie ``JuMP`` , mit dem Sie einfach Optimierungsprogramme formulieren können. Sowie ``GLPK`` einen open-source Solver zum Lösen der Problem und ``StatsPlots`` zur Visualisierung der Lösung.

In [38]:
Pkg.add("JuMP")
Pkg.add("GLPK");
Pkg.add("StatsPlots");

 Resolving package versions...
  Updating `C:\Users\Lotte\.julia\environments\v1.2\Project.toml`
 [no changes]
  Updating `C:\Users\Lotte\.julia\environments\v1.2\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Lotte\.julia\environments\v1.2\Project.toml`
 [no changes]
  Updating `C:\Users\Lotte\.julia\environments\v1.2\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Lotte\.julia\environments\v1.2\Project.toml`
 [no changes]
  Updating `C:\Users\Lotte\.julia\environments\v1.2\Manifest.toml`
 [no changes]


Richten Sie die installierten Pakete so ein, dass sie im folgenden Code verwendet werden können.

In [39]:
using JuMP, GLPK, StatsPlots;

# Capacitated Lot Sizing Problem

### Entscheidungsvariablen:
Bestimmung der **Produktionsmengen** und **Lagerbestände** je Periode sowie der daraus folgenden **Rüstvorgänge**,

### Zielfunktion:
0) **Gesamtkostenminimierung:** $$ \qquad \min z = \displaystyle\sum_{i=1}^I \displaystyle\sum_{t=1}^T(k_i^s\cdot \gamma_{it}+k_i^l\cdot L_{it})$$ 

### Nebenbedingungen:
1) **Kapazitaetsrestriktion:**  $ \hspace{40mm} \displaystyle\sum_{i=1}^I(t_i^p\cdot X_{it}+t_i^s\cdot\gamma_{it} ) \leq c_t \hspace{40mm} \forall t \in T\qquad $ 

2) **Lagerbilanzgleichung:**  $ \hspace{41mm} L_{it} = L_{i,t-1}+X_{it}-d_{it} \hspace{44mm} \forall t \in T, \quad i \in I$

3) **Anfangslagerbestand:** $ \hspace{41mm} L_{i,0} = l_{start} \hspace{66mm} \forall  i \in I$

4) **Rüstbedingung:**:  $ \hspace{50mm} X_{it} \leq M \cdot \gamma_{it} \hspace{63mm} \forall t \in T, \quad i \in I $

5) **Nichtnegativitaetsbedingungen:**  $ \hspace{23mm} X_{it}, L_{it} \geq 0  \hspace{66mm} \forall t \in T, \quad i \in I $

6) **Binaerbedingung:** 
 $ \hspace{48mm}  \gamma_{it} \in \{0,1\}  \hspace{67mm} \forall t \in T, \quad i \in I $
* * *

## Verwendete Symbole

### Mengen

$i \in (1,..,I) \hspace{20mm}$ Produkte
 
$t \in (1,..,T) \hspace{20mm}$ Perioden 


### Parameter

$k_i^l \hspace{38mm}$ Lagerkostensatz

$k_i^s \hspace{38mm}$  Ruestkostensatz  

$t_i^p \hspace{39mm}$  Produktionszeit 

$t_i^s \hspace{39mm}$  Rüstzeit 

$M \hspace{37mm}$  Große Zahl   

$c_t \hspace{38mm}$  Periodenkapazität 

$b_{it}\hspace{37mm}$ Bedarf 

$l_{start} \hspace{34mm}$  Anfangslagerbestand 


### Variablen

$X_{it}$  $\geq0 \hspace{28mm}$       Produktionsmenge   

$L_{it}$  $\geq0 \hspace{28mm}$      Lagermenge         

$\gamma_{it}$  $\in\{0,1\} \hspace{21mm}$ binäre Rüstvariable        

* * *

## Erstellen Sie das Modell namens ``m`` und geben Sie als zu verwendenden Solver GLPK an.
---

In [40]:
m = Model(with_optimizer(GLPK.Optimizer));

### Mengen und Parameter

Fügen Sie die Mengen ein.


In [41]:
#Mengen
Produkte = ["i1", "i2","i3", "i4", "i5"];
Perioden = ["t1","t2","t3","t4", "t5", "t6"];


#Längen
I = length(Produkte);
T = length(Perioden);


Fügen Sie die Parameter ein.

In [42]:
kl = [3, 5, 6, 4, 3];#Lagerkostensatz der Produkte

ks = [20, 50, 40, 30, 50]; #Rüstkostensatz der Produkte
# ks = [2000, 5000, 4000, 3000, 5000];

ts =  [30, 100, 50, 40, 40];#Rüstzeiten der Produkte

tp =  [1, 2, 1, 4, 2]; #Stückbearbeitungszeiten der Produkte

c = [800, 800, 800, 800, 800, 800]; #Periodenkapazität
# c = [400, 400, 400, 400, 400, 400]

M = 1000; #Große Zahl

l_start = [0,0,0,0,0]; #Anfangslagerbestand

      #t1. t2, t3, t4,  t5,  t6
b = [  10  25  30  100   0  130   #Produkt 1
        0   5  40    0  10   60   #Produkt 2
        5  45  30    0  40   60   #Produkt 3
        0  40  20   15  80    0   #Produkt 4
       20   0   5   15  70   50]; #Produkt 5

### Entscheidungsvariablen
---
Definieren Sie die Entscheidungsvariablen. Achten Sie auf die Definitionsbereiche (Nebenbedingungen 5 und 6)

5) **Nichtnegativitätsbedingungen**: Produktionsmengen und Lagerbestände dürfen nicht negativ werden.

 $ \qquad X_{it}, L_{it} \geq 0  \qquad\qquad\qquad\qquad\qquad\qquad \forall t \in T, \quad i \in I $

In [43]:
@variable(m,X[1:I,1:T] >= 0,Int);  #Produktionsmenge von Produkt i in Periode t

In [44]:
@variable(m,L[1:I,0:T] >= 0,Int);  #Lagerbestand von Produkt i am Periodenende von t

6) **Binaerbedingung**: Rüstvorgänge werden immer ganz ($\delta_{it} = 1$) oder gar nicht ($\delta_{it} = 0$) ausgeführt. Die binäre Rüstvariable nimmt also entweder den Wert 1 oder 0 an.

 $ \qquad  \gamma_{it} \in \{0,1\}  \qquad\qquad\qquad\qquad\qquad\qquad \forall t \in T, \quad i \in I $

In [45]:
@variable(m,gamma[1:I,1:T],Bin); #Die binäre Rüstvariable von Produkt i in Periode t

## Zielfunktion

0) **Gesamtkostenminimierung:** Die Gesamtkosten sollen minimiert werden. Sie bestehen aus der Summe von Rüstkosten und Lagerkosten aller Produkte über alle Perioden. Die Rüstkosten berechnen sich hierbei durch Multipizieren der Anzahl der Rüstvorgänge (Summe aler binären Rüstvariablen) mit dem Rüstkostensatz, die Lagerkosten durch Multiplizieren der eingelagerten Menge mit dem Lagerkostensatz.

$ \qquad \min z = \displaystyle\sum_{i=1}^I \displaystyle\sum_{t=1}^T(k_i^s\cdot \gamma_{it}+k_i^l\cdot L_{it})$ 


In [46]:
@objective(m, Min,  sum( ks[i] * gamma[i,t] +  kl[i] * L[i,t]  for i=1:I for t=1:T) );

## Nebenbedingungen

1) **Kapazitaetsrestriktion:** Die Summe aus Produktions- und Rüstzeit aller Produkte darf in jeder Periode die vorhandene Kapazität der Ressource j nicht überschreiten.

 $$ \qquad \displaystyle\sum_{i=1}^I(t_i^p\cdot X_{it}+t_i^s\cdot\gamma_{it} ) \leq c_t \hspace{40mm} \forall t \in T\qquad $$ 
 


In [47]:
@constraint(m, KapRes[t=1:T], sum((ts[i] * gamma[i,t] + tp[i] * X[i,t]) for i=1:I) <= c[t] );

2) **Lagerbilanzgleichung**: Der Lagerbestand am Ende einer Periode berechnet sich aus der Summe der Produktionsmenge und des Lagerbestandes der Vorperiode abzüglich der Absatzmenge (Nachfrage).

 $$ \qquad L_{it} = L_{i,t-1}+X_{it}-b_{it} \hspace{40mm} \forall t \in T, \quad i \in I$$

In [48]:
@constraint(m, Lager[i=1:I,t=1:T], L[i,t] == L[i,t-1] + X[i,t]  - b[i,t]);

 3) **Anfangslagerbestand**: Der Anfangslagerbestand aller Produkte entspricht dem initial gesetzen $l_i$.
 
  $$ \qquad L_{i,0} = l_{start} \hspace{40mm} \forall  i \in I$$


In [49]:
@constraint(m, AnfLager[i=1:I], L[i,0] == l_start[i]);

4) **Rüstbedingung**: Wenn für ein Produkt in einer Periode nicht gerüstet wird, ist die produzierte Menge dieses Produkts in dieser Periode 0. Wenn für ein Produkt in einer Periode gerüstet wird, wird die produzierte Menge durch die Rüstbedingung nicht eingeschränkt.

 $$ \qquad X_{it} \leq M \cdot \gamma_{it} \hspace{40mm} \forall t \in T, \quad i \in I $$

In [50]:
@constraint(m, Ruestbed[i=1:I,t=1:T], X[i,t] <= M * gamma[i,t]);

## Lösen Sie das Modell.
---

In [51]:
optimize!(m)

Lassen Sie sich den Zielfunktionswert Z anzeigen.

In [52]:
println("Objective value Z: ", JuMP.objective_value(m))

Objective value Z: 31930.0


Lassen Sie sich die Produktionsmengen, die Lagermengen und die Rüstvariable anzeigen.

Produktionsmenge von Produkt i in Periode t

In [53]:
JuMP.value.(X)

5×6 Array{Float64,2}:
  65.0    0.0  0.0  230.0   0.0  0.0
   0.0  115.0  0.0    0.0   0.0  0.0
 180.0    0.0  0.0    0.0   0.0  0.0
   0.0   75.0  0.0    0.0  80.0  0.0
 160.0    0.0  0.0    0.0   0.0  0.0

Lagerbestand von Produkt i am Ende von Periode t

In [54]:
JuMP.value.(L)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 0:6
And data, a 5×7 Array{Float64,2}:
 0.0   55.0   30.0    0.0  130.0  130.0  0.0
 0.0    0.0  110.0   70.0   70.0   60.0  0.0
 0.0  175.0  130.0  100.0  100.0   60.0  0.0
 0.0    0.0   35.0   15.0    0.0    0.0  0.0
 0.0  140.0  140.0  135.0  120.0   50.0  0.0

Binäre Rüstvariable von Produkt i in Periode t

In [55]:
JuMP.value.(gamma)

5×6 Array{Float64,2}:
 1.0  0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0